In [ ]:
# --- IMPORTS ---
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import os
import random


# --- CONDITION MAPPING ---
nail_condition_rules = {
    "Bluish Nail": "Possible oxygen deficiency or cyanosis; may indicate heart or lung problems.",
    "Clubbing": "Possible chronic lung or heart disease causing low oxygen levels.",
    "Onychomycosis": "Possible fungal nail infection; causes thick, brittle, or discolored nails.",
    "Psoriasis": "Possible autoimmune condition causing pitting or discoloration on nails.",
    "Healthy Nail": "Normal nail appearance; no visible abnormalities or diseases.",
    "Acral Lentiginous Melanoma": "Possible serious skin cancer forming under the nail; seek medical attention.",
    "Onychogryphosis": "Possible nail thickening or curvature; common in elderly or due to trauma.",
    "Pitting": "Possible psoriasis or eczema; small depressions or dents on the nail surface.",
    "Yellow Nail": "Possible fungal infection, liver disease, or chronic respiratory issue.",
    "White Nail": "Possible liver disease, low protein levels, or anemia.",
    "Beau’s Lines": "Possible history of illness, infection, or stress disrupting nail growth.",
    "Koilonychia": "Possible iron deficiency anemia; nail appears spoon-shaped."
}


# --- LOAD MODEL FROM JSON + WEIGHTS ---
with open("saved_models/model_architecture_bestrun.json", "r") as json_file:
    model_json = json_file.read()


model = model_from_json(model_json)
model.load_weights("saved_models/model_weights_bestrun.weights.h5")


print("✅ Model loaded successfully for prediction.")


# --- PREPARE TEST DATA ---
test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_directory(
    "dataset/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


# --- PREDICTION ---
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes


# --- GET CLASS LABELS ---
class_labels = list(test_generator.class_indices.keys())


# --- PRINT SUMMARY ---
print(f"\nFound {test_generator.samples} images belonging to {len(class_labels)} classes.")


# --- PICK 5 RANDOM SAMPLES ---
random.seed(42)
sample_indices = random.sample(range(len(predicted_classes)), 5)


# --- PRINT HEADER ---
print("\n{:<75} {:<30} {:<30} {}".format(
    "FILENAME", "TRUE LABEL", "PREDICTED", "DESCRIPTION"
))
print("-" * 170)


correct_count = 0


# --- PRINT EACH SAMPLE WITH DESCRIPTION ---
for idx in sample_indices:
    filename = test_generator.filenames[idx]
    true_label = class_labels[true_labels[idx]]
    pred_label = class_labels[predicted_classes[idx]]
    description = nail_condition_rules.get(pred_label, "No description available.")
   
    if true_label == pred_label:
        correct_count += 1


    print("{:<75} {:<30} {:<30} {}".format(
        filename, true_label, pred_label, description
    ))


print(f"\n✅ Correct predictions out of 5 random samples: {correct_count} / 5")
print(f"✅ Random 5 images displayed out of {len(test_generator.filenames)} total.")


# --- ACCURACY & CONFUSION MATRIX ---
accuracy = accuracy_score(true_labels, predicted_classes)
cm = confusion_matrix(true_labels, predicted_classes)


print(f"\nTest Accuracy: {accuracy * 100:.2f}%")
print("\nConfusion Matrix:")
print(cm)


FILENAME                                                                    TRUE LABEL                     PREDICTED                      DESCRIPTION
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Acral Lentiginous Melanoma\45Screenshot 2025-10-26 003109.png               Acral Lentiginous Melanoma     Onychogryphosis                Possible nail thickening or curvature; common in elderly or due to trauma.
Onychogryphosis\Screenshot 2025-10-25 211836.png                            Onychogryphosis                Onychogryphosis                Possible nail thickening or curvature; common in elderly or due to trauma.
White Nail (Leukonychia)\Screenshot 2025-10-10 at 3.43.14 PM.png            White Nail (Leukonychia)       Onychomycosis                  Possible fungal nail infection; causes thick, brittle, or discolored nails.
Healthy Nail\Screenshot 2025-10-25 224